In [1]:
import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.csv
import pyarrow.compute
import pyarrow.parquet as pq
import pytz
from tqdm.auto import tqdm

In [2]:
schema_in = pa.schema([
    ("species", pa.string()),
    ("animalID", pa.string()),
    ("collarID", pa.string()),
    ("date_time_local", pa.timestamp("us")),
    ("lon", pa.float64()),
    ("lat", pa.float64()),
    ("tz", pa.string()),
    ("source", pa.string()),
    ("trackID", pa.string()),
])

schema_out = pa.schema([
    ("timestamp", pa.timestamp("us")),
    ("location_lat", pa.float64()),
    ("location_long", pa.float64()),
    ("study_id", pa.string()),
    ("individual_id", pa.string()),
    ("deployment_id", pa.string()),
    ("tag_id", pa.string()),
#     ("section", pa.string())
])

In [3]:
table = pa.csv.read_csv("all.csv",
                        convert_options=pa.csv.ConvertOptions(
                            column_types=schema_in,
                            timestamp_parsers=["%Y-%m-%dT%H:%M:%SZ"]
                        ))

In [4]:
table_df = table.to_pandas()

In [5]:
def timestamps_to_utc(df):
    assert df["tz"].nunique() == 1
    try:
        df["timestamp"] = df["date_time_local"].dt.tz_localize(df["tz"].iloc[0], ambiguous="infer")
    except pytz.AmbiguousTimeError:
        # remove ambiguous ones if infer failed
        df["timestamp"] = df["date_time_local"].dt.tz_localize(df["tz"].iloc[0], ambiguous="NaT")
        df = df.query("not timestamp.isna()")
    df["timestamp"] = df["timestamp"].dt.tz_convert("UTC").dt.tz_localize(None)
    return df

table_df = table_df.groupby("tz").apply(timestamps_to_utc).reset_index(level="tz", drop=True).sort_index()

In [6]:
table_df = table_df.query("source.str.contains('simon') and not lon.isna() and not lat.isna()")

In [7]:
table_out_df = pd.DataFrame.from_dict({
    "timestamp": table_df["timestamp"],
    "location_lat": table_df["lat"],
    "location_long": table_df["lon"],
    "study_id": table_df["source"].str.cat(table_df["site"], sep=" - "),
    "individual_id": table_df["animalID"],
    "deployment_id": table_df["trackID"],
    "tag_id": table_df["collarID"],
})

In [8]:
table_out = pa.Table.from_pandas(table_out_df, preserve_index=False, schema=schema_out)

In [9]:
table_out = table_out.sort_by([("individual_id", "ascending"),
                               ("deployment_id", "ascending"),
                               ("timestamp", "ascending")])

In [10]:
with pq.ParquetWriter("locations_simon.parquet", schema=schema_out) as writer:
    writer.write_table(table_out)

In [11]:
# Convert metadata about individuals to a parquet file
schema_out_indiv = pa.schema([
    ("id", pa.string()),
    ("species", pa.string())
])

indiv_df = table_df[["animalID", "species"]].rename(columns={"animalID": "id"}).drop_duplicates()
indiv_table = pa.Table.from_pandas(indiv_df, preserve_index=False, schema=schema_out_indiv)

with pq.ParquetWriter("individuals_simon.parquet", schema=schema_out_indiv) as writer:
    writer.write_table(indiv_table)

In [12]:
# Check that each animalID has exactly one unique species
(table_df.groupby("animalID").agg({"species": "nunique"}) == 1).all().all()

True